# Search Functions
> Functions to search across all library components:

In [ ]:
#| default_exp cli.search

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Tuple, Any, Optional
from dataclasses import dataclass
from cjm_fasthtml_tailwind.cli.utils import (
    SearchResult, search_in_name_and_text, 
    search_in_fields, discover_utility_modules, list_utility_modules, search_in_text
)
from cjm_fasthtml_tailwind.cli.factory_extraction import list_all_factories
from cjm_fasthtml_tailwind.cli.example_discovery import list_all_examples
from cjm_fasthtml_tailwind.cli.helper_discovery import get_module_helpers

In [ ]:
#| export
def search_factories(
    query: str,  # Search query
    include_source: bool = False,  # Whether to search in source code
    case_sensitive: bool = False  # Whether to perform case-sensitive search
) -> List[SearchResult]:  # List of search results
    """Search in all factories."""
    results = []
    
    for module_name, factories in list_all_factories().items():
        for factory_info in factories:
            # Use helper function for name and documentation search
            results.extend(search_in_name_and_text(
                query=query,
                item_name=factory_info.name,
                text=factory_info.doc,
                content_type='factory',
                module_name=module_name,
                text_location='documentation',
                case_sensitive=case_sensitive
            ))
    
    return results

In [ ]:
#| export
def search_examples(
    query: str,  # Search query
    include_source: bool = False,  # Whether to search in source code
    case_sensitive: bool = False  # Whether to perform case-sensitive search
) -> List[SearchResult]:  # List of search results
    """Search in all test examples."""
    results = []
    
    for module_name, examples in list_all_examples().items():
        for example_info in examples:
            # Define fields to search
            fields = {
                'name': lambda e: e.name,
                'feature': lambda e: e.feature,
                'docstring': lambda e: e.docstring
            }
            
            # Search in fields
            results.extend(search_in_fields(
                item=example_info,
                query=query,
                fields=fields,
                content_type='example',
                module_name=module_name,
                item_name=example_info.name,
                case_sensitive=case_sensitive
            ))
            
            # Search in source code if requested
            if include_source:
                source_result = search_in_source_code(
                    source=example_info.source,
                    query=query,
                    content_type='example',
                    module_name=module_name,
                    item_name=example_info.name,
                    case_sensitive=case_sensitive
                )
                if source_result:
                    results.append(source_result)
    
    return results

In [ ]:
#| export
def search_helpers(
    query: str,  # Search query
    include_source: bool = False,  # Whether to search in source code
    case_sensitive: bool = False  # Whether to perform case-sensitive search
) -> List[SearchResult]:  # List of search results
    """Search in all helper functions."""
    results = []
    
    # Get helpers for all modules
    for module_name, module in discover_utility_modules():
        helpers = get_module_helpers(module_name)
        
        for helper_info in helpers:
            # Define fields to search
            fields = {
                'name': lambda h: h.name,
                'signature': lambda h: h.signature,
                'docstring': lambda h: h.docstring
            }
            
            # Search in fields
            results.extend(search_in_fields(
                item=helper_info,
                query=query,
                fields=fields,
                content_type='helper',
                module_name=module_name,
                item_name=helper_info.name,
                case_sensitive=case_sensitive
            ))
            
            # Search in source code if requested
            if include_source:
                source_result = search_in_source_code(
                    source=helper_info.source,
                    query=query,
                    content_type='helper',
                    module_name=module_name,
                    item_name=helper_info.name,
                    case_sensitive=case_sensitive
                )
                if source_result:
                    results.append(source_result)
    
    return results

In [ ]:
#| export
def search_modules(
    query: str,  # Search query
    case_sensitive: bool = False  # Whether to perform case-sensitive search
) -> List[SearchResult]:  # List of search results
    """Search in module names and documentation."""
    results = []
    
    for module_name, doc in list_utility_modules().items():
        # Search in module name
        if search_in_text(query, module_name, case_sensitive):
            results.append(SearchResult(
                content_type='module',
                module_name=module_name,
                item_name=module_name,
                match_context=module_name,
                match_location='name'
            ))
        
        # Search in module documentation
        if search_in_text(query, doc, case_sensitive):
            results.append(SearchResult(
                content_type='module',
                module_name=module_name,
                item_name=module_name,
                match_context=doc,
                match_location='documentation'
            ))
    
    return results

In [ ]:
#| export
def search_all(
    query: str,  # Search query
    content_types: Optional[List[str]] = None,  # Types to search in ('factories', 'examples', 'helpers', 'modules')
    include_source: bool = False,  # Whether to search in source code
    case_sensitive: bool = False  # Whether to perform case-sensitive search
) -> List[SearchResult]:  # List of all search results
    """Search across all content types."""
    if content_types is None:
        content_types = ['factories', 'examples', 'helpers', 'modules']
    
    all_results = []
    
    if 'factories' in content_types:
        all_results.extend(search_factories(query, include_source, case_sensitive))
    
    if 'examples' in content_types:
        all_results.extend(search_examples(query, include_source, case_sensitive))
    
    if 'helpers' in content_types:
        all_results.extend(search_helpers(query, include_source, case_sensitive))
    
    if 'modules' in content_types:
        all_results.extend(search_modules(query, case_sensitive))
    
    # Remove duplicates (same item found in multiple locations)
    seen = set()
    unique_results = []
    for result in all_results:
        key = (result.content_type, result.module_name, result.item_name, result.match_location)
        if key not in seen:
            seen.add(key)
            unique_results.append(result)
    
    return unique_results

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()